# Gaia Globular Cluster Orbits
from here: https://www.astro.rug.nl/~ahelmi/research/dr2-dggc/gaia-dr2-dggc-2.pdf with past orbits calculated using astro-gala

### Imports and function definitions

In [139]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib.request, math, copy
sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"), os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod
from config import Settings


### USES Conf Template

In [140]:
Template = """mesh
{
    data  gcOrbits ./modules/$folderName/data/Gaia_GCs.raw
    dataHints gcOrbits disablePostprocessing
    glslVersion 330 
    enableTypeCheck true
    
    propertyCollection
    {
        __objectName__
        {
            vec1f trailLength 25
            vec1f trailWidth 50.0
            vec4f trailColor 0.4 0.1\5 1.0 1.0
        }
    }

    pass
    {
        useDataObject gcOrbits
        shader
        {
            type defaultMeshShader
            {
                StateManagerVar __objectName__.trailLength trailLength
                StateManagerVar __objectName__.trailWidth trailWidth
                StateManagerVar __objectName__.trailColor trailColor
                StateManagerVar __objectName__.Alpha alpha
                vertexShader ./modules/$folderName/shaders/vertex.glsl
                fragmentShader ./modules/$folderName/shaders/trails_fragment.glsl
                geometryShader ./modules/$folderName/shaders/trails_geometry.glsl
            }
        }
    }

}"""

### Gaia GC Class

In [141]:
class GaiaGCs():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("shaders\\trails_geometry.glsl",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("shaders\\trails_fragment.glsl",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("shaders\\vertex.glsl",dataFolder)
        self.Alpha = 0.75
    def setAlpha(self,Alpha):
        self.Alpha = Alpha
    def generatemod(self):
        self.object.setgeometry(self.object.name+".usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+".usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [142]:
model = GaiaGCs(uvmod.OrbitalObject())
generator = uvmod.Generator()
scene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()

### Specify Settings and generate the module

In [143]:
model.object.setcameraradius(1.0)
model.object.setcoord("MilkyWay")
model.object.setname("GaiaGCs")
model.object.setguiname("/Milky Way/Gaia/Globular Cluster Orbits")
model.object.settargetradius(10)
model.object.setscalefactor(1.0)
model.object.showatstartup(False)
modinfo.setname("Gaia Globular Cluster Orbits")
modinfo.setauthor("Mark SubbaRao<br />Adler Planetarium")
modinfo.cleardependencies()
modinfo.setdesc("Gaia Globular Cluster Orbits from Helmi et. al 2018")
modinfo.setthumbnail("data/images/thumbnail.png")
modinfo.setversion("1.0")

generator.generate("GaiaGCs",[],[model],modinfo)

### Helper Functions for modifing code
##### Reload Module and Shaders in Uniview

In [6]:
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")

##### Copy modified Shader files and reload

In [123]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder+'\\shaders', Settings.uvcustommodulelocation+'\\'+model.object.name+'\\shaders')
uvmod.Utility.senduvcommand(model.object.name+".reload; system.reloadallshaders")